# Running LSPI on the UR5

In [1]:
from __future__ import print_function
import rosgym
import gym
import numpy as np
import os
import rospy
import imp
import random
import requests
import time
import yaml
from os import path

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.append(os.path.dirname(os.getcwd()) + "/rl_agents/python")

In [4]:
import basic_agents
from geometry_msgs.msg import PointStamped
from visualization_msgs.msg import Marker

In [5]:
def make_environment(robot_conf_file, planning_group, num_joints, z_limit,
                     x_limits, y_limits, **kwargs):
    """
    Creates an OpenAI Gym environment corresponding to the supplied robot
    configuration file, and configuration options.

    :return: rosgym environment
    """

    env = rosgym.make_randomgoal_robot_env(
        "ur5_config_random_goal", robot_conf_file, planning_group, num_joints,
        z_limit, x_limits[0], x_limits[1], y_limits[0], y_limits[1])
    return env

class SimpleConfig(object):
    def __init__(self, config, defaults):
        self._config = config
        self._defaults = defaults

    def __getitem__(self, key):
        return self._config.get(key, self._defaults.get(key))

In [6]:
defaults = {
    "robot_conf_file":
    "/home/cannon/rl_wksp/src/rosgym/src/rosgym/ur5_config_random_goal.py",
    "planning_group":
    "manipulator",
    "num_joints":
    3,
    "z_limit":
    2.0,
    "x_min":
    -2.0,
    "x_max":
    2.0,
    "y_min":
    -2.0,
    "y_max":
    2.0,
    "num_tests":
    10,
    "num_episodes":
    100,
}

In [7]:
sc = SimpleConfig({}, defaults)

# make our random agent
print("Attempting to make the rosgym environment...")
ur5env = make_environment(sc["robot_conf_file"], sc["planning_group"],
                       sc["num_joints"], sc["z_limit"], [sc["x_min"], sc["x_max"]],
                       [sc["y_min"], sc["y_max"]])
print("Made the rosgym environment...")

Attempting to make the rosgym environment...


Connecting to control group manipulator
Found move groups: ['endeffector', 'forearm', 'manipulator', 'shoulder', 'upper_arm', 'wrist_1', 'wrist_2']
Found planning scene: <moveit_commander.planning_scene_interface.PlanningSceneInterface object at 0x7f8cdac18350>


Connecting reset handler
Finished connecting wt_commander
Set up MoveGroupInterface
[INFO] [1554974447.056666, 46.088000]: Waiting for get_planning_scene
Set up PlanningSceneInterface
Initialized SimpleActionClient


Finished connecting reset handler
Waiting for sensor /joint_states
Got initial data for sensor /joint_states
RobotAgent_UR5 has been initialized.
Finished initializing robot
Initialized sensors
Connecting to control group manipulator


Finished waiting


Initialized move groups and joints
Initialized service proxies
Getting new goal
Sampling new goal
Goal is: [-0.3054623372679364, 0.18661042552733695, 0.5351967249896288]


Finding new goal pose
Finding new goal pose
Finding new goal pose
Found goal pose: x: -0.305462337268
y: 0.186610425527
z: 0.53519672499
Made the rosgym environment...


In [8]:
ur5env.action_space.spaces[0].low

array([-6.2831853, -6.2831853, -6.2831853])

In [9]:
ur5env.action_space.spaces[0].high

array([ 6.2831853,  6.2831853,  6.2831853])

In [10]:
ur5env.observation_space

Tuple(Box(6,), Box(6,))

So the low and high positions of our action space are $-2\pi$ and $2\pi$.
A good start for working on the discretization would probably just be to 
uniformly chunk our action space.

In [11]:
# this discretizer does that
from lspi.util import ur5_discretizer

In [12]:
actions = ur5_discretizer(ur5env.action_space, 4)

In [13]:
actions[0]

(array([-6.2831853, -6.2831853, -6.2831853]),)

## generating sample data

We need to generate our sample data after we've already wrapped
this space in a discrete action environment.

In [14]:
from lspi.util import DiscreteEnvWrapper

env = DiscreteEnvWrapper(ur5env, actions)

In [16]:
from lspi.policy import RandomPolicy
from lspi.util import SampleGenerator

rp = RandomPolicy(env)
data = SampleGenerator(env, rp).sample(10, 1000)

Planning to [1.5707963267948966, 0.7853981633974483, 0, 0, 0, 0]  on joints  ['elbow_joint', 'shoulder_lift_joint', 'shoulder_pan_joint', 'wrist_1_joint', 'wrist_2_joint', 'wrist_3_joint']
Failed to plan to trajectory!
Retrying Reset...
Planning to [1.5707963267948966, 0.7853981633974483, 0, 0, 0, 0]  on joints  ['elbow_joint', 'shoulder_lift_joint', 'shoulder_pan_joint', 'wrist_1_joint', 'wrist_2_joint', 'wrist_3_joint']
Successfully planned to position. Executing trajectory.


Sampling new goal


Finding new goal pose
Found goal pose: x: 0.0163792327277
y: -0.38737962413
z: 0.761430410219
New target pose: [0.016379232727700317, -0.38737962413025107, 0.7614304102187317]
Planning to [1.5707963267948966, 0.7853981633974483, 0, 0, 0, 0]  on joints  ['elbow_joint', 'shoulder_lift_joint', 'shoulder_pan_joint', 'wrist_1_joint', 'wrist_2_joint', 'wrist_3_joint']
Successfully planned to position. Executing trajectory.


Sampling new goal


Finding new goal pose
Finding new goal pose
Finding new goal pose
Finding new goal pose
Finding new goal pose
Found goal pose: x: 0.0335958566342
y: -0.112661960754
z: 0.583514364331
New target pose: [0.033595856634212815, -0.11266196075403795, 0.583514364330558]
Planning to [1.5707963267948966, 0.7853981633974483, 0, 0, 0, 0]  on joints  ['elbow_joint', 'shoulder_lift_joint', 'shoulder_pan_joint', 'wrist_1_joint', 'wrist_2_joint', 'wrist_3_joint']
Successfully planned to position. Executing trajectory.


...

Finding new goal pose
Finding new goal pose
Finding new goal pose
Found goal pose: x: -0.582374285935
y: -0.0427582856051
z: 0.568624442565
New target pose: [-0.582374285934934, -0.042758285605123104, 0.5686244425651003]
Planning to [1.5707963267948966, 0.7853981633974483, 0, 0, 0, 0]  on joints  ['elbow_joint', 'shoulder_lift_joint', 'shoulder_pan_joint', 'wrist_1_joint', 'wrist_2_joint', 'wrist_3_joint']
Failed to plan to trajectory!
Retrying Reset...
Planning to [1.5707963267948966, 0.7853981633974483, 0, 0, 0, 0]  on joints  ['elbow_joint', 'shoulder_lift_joint', 'shoulder_pan_joint', 'wrist_1_joint', 'wrist_2_joint', 'wrist_3_joint']
Failed to plan to trajectory!
Retrying Reset...
Planning to [1.5707963267948966, 0.7853981633974483, 0, 0, 0, 0]  on joints  ['elbow_joint', 'shoulder_lift_joint', 'shoulder_pan_joint', 'wrist_1_joint', 'wrist_2_joint', 'wrist_3_joint']
Successfully planned to position. Executing trajectory.


Sampling new goal


Finding new goal pose
Found goal pose: x: 0.373511634873
y: -0.389739641422
z: 0.203779540179
New target pose: [0.3735116348734456, -0.38973964142174666, 0.2037795401792401]
Planning to [1.5707963267948966, 0.7853981633974483, 0, 0, 0, 0]  on joints  ['elbow_joint', 'shoulder_lift_joint', 'shoulder_pan_joint', 'wrist_1_joint', 'wrist_2_joint', 'wrist_3_joint']
Successfully planned to position. Executing trajectory.


Sampling new goal


Finding new goal pose
Finding new goal pose
Finding new goal pose
Finding new goal pose
Found goal pose: x: 0.278668013844
y: -0.122037503803
z: 0.775028383044
New target pose: [0.27866801384387907, -0.12203750380337779, 0.7750283830438948]


#### basis

For the first try we'll just use quadratic matrices.

In [35]:
statesize = 12
actionsize = 3
size = statesize + actionsize

In [36]:
def randomMatrix(shape, low=-1, high=1):
    return np.random.random(shape) * (high - low) + low

numMat = 30
coefs = [randomMatrix((size, size)) for _ in range(numMat)]

In [37]:
from lspi.basis import DiscreteQuadraticTupleBasis

basis = DiscreteQuadraticTupleBasis([coefs], actions, env.observation_space)

In [38]:
basis(env.observation_space.sample(), env.action_space.sample())

array([   1.        ,  -26.06159623,  -32.68497302,  -60.64137087,
        145.20785957, -122.40088118,  -65.96842271,   70.09445487,
         12.65181081,  -37.24441037,  -70.31080766,    3.45705329,
         21.06265736,  154.11020743,   31.90463303,   74.71831233,
        -88.51430001,  -21.77724507,  -21.55606833,   42.20956175,
        -16.85830779,  106.45596578,   10.99745785,   -9.62722788,
         21.48098453,  -11.63941566,   76.69937712,   13.03554931,
         14.22401432,  -41.08521   ,  -36.97305414])

In [40]:
from lspi.optim import LSTDQ, LSPI
from lspi.policy import DiscreteActionBasisPolicy

lstdq = LSTDQ(basis, discount=0.95)
dabp = DiscreteActionBasisPolicy(env.action_space, basis, np.zeros(basis.rank))
lspi = LSPI(lstdq, dabp, max_iter=50, epsilon=1e-3, verbose=True)

In [41]:
lspi.fit(data)

Iteration 0: Diff: 2193.32924933
Iteration 1: Diff: 1912.37397313
Iteration 2: Diff: 17.6683018571
Iteration 3: Diff: 0.0616077814138


In [49]:
rewards = []
actions = []

for episode in range(10):
    state = env.reset()
    done = False
    cumr = 0
    itr = 0
    while not done and itr < 500:
        action = lspi.policy(state)
        state, reward, done, _ = env.step(action)
#         done = cumr < -300 or state[0] > 0.5
        cumr += reward
#         env.render()
#     print(cumr)
        print("Action: {}, State: {}, Reward: {}".format(action, state, reward))
    rewards.append(cumr)
#     print("Episode {episode}, Reward: {reward}".format(episode=episode, reward=reward))
rewards = np.array(rewards)

Planning to [1.5707963267948966, 0.7853981633974483, 0, 0, 0, 0]  on joints  ['elbow_joint', 'shoulder_lift_joint', 'shoulder_pan_joint', 'wrist_1_joint', 'wrist_2_joint', 'wrist_3_joint']
Successfully planned to position. Executing trajectory.


Sampling new goal


Finding new goal pose
Found goal pose: x: -0.210415708346
y: -0.0938964763398
z: 0.42216868985
New target pose: [-0.21041570834617976, -0.09389647633980201, 0.4221686898498746]
Action: 25, State: ([1.5534821478771024, 0.8000265052334683, 0.018917359273430634, 0.10491482655739465, -0.7318237526005614, -0.6031945969838799, -0.7244898863215887, -0.21251038048432008, 0.37959324202988953, -0.21041570834617976, -0.09389647633980201, 0.4221686898498746],), Reward: -14.0045389439
Action: 25, State: ([1.556683485934367, 0.7975937785182774, 0.014294236350371747, 0.1375606027745576, -0.8305710842286973, -0.9490648352190111, -0.7239212923997657, -0.2103982984904198, 0.3799237733573196, -0.21041570834617976, -0.09389647633980201, 0.4221686898498746],), Reward: -14.0036712699
Action: 25, State: ([1.5453462552691342, 0.7974148014485252, -0.012026742525288903, -0.8110748659460737, -0.6062371614397102, -1.8996954760165852, -0.7313729982282571, -0.1967886493903625, 0.378067985302112, -0.2104157083461797

Action: 25, State: ([1.4131098207173451, 0.8588990846597566, -0.45216046121762243, -0.491681747867196, -0.3733846598163907, -1.2037778058328579, -0.7621117566327291, 0.05273019508689043, 0.3806600431975968, -0.21041570834617976, -0.09389647633980201, 0.4221686898498746],), Reward: -14.0129927866
Action: 25, State: ([1.4083581955278905, 0.8618391439681448, -0.4590738834546082, -0.48894579106536923, -0.37549692491044917, -1.1366529855912562, -0.7621117566327291, 0.05273019508689043, 0.3806600431975968, -0.21041570834617976, -0.09389647633980201, 0.4221686898498746],), Reward: -14.0129927866
Action: 25, State: ([1.3983967976501654, 0.8655170551663414, -0.470621386091878, -0.505866953011621, -0.3571566882205671, -0.9068842858817442, -0.7629125038734283, 0.06104151677651552, 0.3813159886165669, -0.21041570834617976, -0.09389647633980201, 0.4221686898498746],), Reward: -14.0145534591
Action: 25, State: ([1.3890337617325583, 0.8705031902344471, -0.48465313374109975, -0.49622622586294296, -0.3

Action: 25, State: ([1.1927511175011771, 0.9561859020724537, -0.6952629106310386, -0.28422551093757215, -0.45689866803589874, -0.7073550982567401, -0.7601215233613235, 0.21907065301240836, 0.3817125997337067, -0.21041570834617976, -0.09389647633980201, 0.4221686898498746],), Reward: -14.0378125243
Action: 25, State: ([1.1892031826019016, 0.9578237496348105, -0.6975356018168171, -0.2583688910656301, -0.4794862197062648, -0.4200161566113296, -0.7596520252300873, 0.22399587893081757, 0.38171223044610575, -0.21041570834617976, -0.09389647633980201, 0.4221686898498746],), Reward: -14.0386480162
Action: 25, State: ([1.1810176372196581, 0.9615274161799743, -0.7066090823517346, -0.33051299526605027, -0.4539603928731446, -0.7066913952609936, -0.7596520252300873, 0.22399587893081757, 0.38171223044610575, -0.21041570834617976, -0.09389647633980201, 0.4221686898498746],), Reward: -14.0386480162
Action: 25, State: ([1.1734466539750112, 0.9658485394543952, -0.7139152441203702, -0.25800376855384555, 

...

Action: 25, State: ([-2.0978017696302587, 1.6511045431608418, -2.084780609412464, 0.6898272472060772, -0.8012715187720791, -0.5034977016314763, -0.13883657326534055, 0.25578583601276494, -0.027578316818000048, -0.21041570834617976, -0.09389647633980201, 0.4221686898498746],), Reward: -13.4518122925
Action: 25, State: ([-2.097444068091151, 1.6513876134599341, -2.084085900961659, 0.7198610057487012, -0.8298326053963454, -0.39431665120616805, -0.13861202095267125, 0.25576685509166747, -0.027324645577650186, -0.21041570834617976, -0.09389647633980201, 0.4221686898498746],), Reward: -13.4516652712
Action: 25, State: ([-2.097125366689699, 1.6511614957401815, -2.0840110977061217, 0.7250673456003986, -0.8391546034409614, -0.38963922784624505, -0.13861202095267125, 0.25576685509166747, -0.027324645577650186, -0.21041570834617976, -0.09389647633980201, 0.4221686898498746],), Reward: -13.4516652712
Action: 25, State: ([-2.097607143567762, 1.6516367116884156, -2.0834466399992895, 0.694320171657395

Action: 25, State: ([-2.097723484091164, 1.6516525009757386, -2.0832524181500363, 0.68619780811511, -0.8353805147137313, -0.3313397430509473, -0.13876373498183156, 0.2557211923148547, -0.02736991358811075, -0.21041570834617976, -0.09389647633980201, 0.4221686898498746],), Reward: -13.4517015493
Action: 25, State: ([-2.09800942085678, 1.6512302498760034, -2.084599410005019, 0.6869809316357625, -0.807694110113076, -0.507329546776792, -0.13829989105833487, 0.2557768357810696, -0.02758761847173319, -0.21041570834617976, -0.09389647633980201, 0.4221686898498746],), Reward: -13.4515507926
Action: 25, State: ([-2.097034562299405, 1.6512146376597832, -2.084665280124919, 0.7464087420836135, -0.8172524101516401, -0.4529160514803473, -0.13829989105833487, 0.2557768357810696, -0.02758761847173319, -0.21041570834617976, -0.09389647633980201, 0.4221686898498746],), Reward: -13.4515507926
Action: 25, State: ([-2.0973000826147405, 1.6511978654380304, -2.083783013818806, 0.7071998836484832, -0.84387730

Action: 25, State: ([-2.097171593726724, 1.65132119295248, -2.0836844047161662, 0.7346559474470366, -0.8440859835624153, -0.38270945239926396, -0.13845319996646854, 0.2557024482263365, -0.02738105302410493, -0.21041570834617976, -0.09389647633980201, 0.4221686898498746],), Reward: -13.4515388545
Action: 25, State: ([-2.0972989707337604, 1.6512570487692768, -2.083510305027181, 0.7197512035548816, -0.8429620137390977, -0.386805881381842, -0.1387846513646557, 0.25573841398702546, -0.02768232841237256, -0.21041570834617976, -0.09389647633980201, 0.4221686898498746],), Reward: -13.4517559752
Action: 25, State: ([-2.097410584069486, 1.6517555005023583, -2.0847565511153903, 0.7273920748889018, -0.7896901171628294, -0.5103624287849016, -0.1387846513646557, 0.25573841398702546, -0.02768232841237256, -0.21041570834617976, -0.09389647633980201, 0.4221686898498746],), Reward: -13.4517559752
Action: 25, State: ([-2.0974398667467486, 1.651277972798117, -2.084679758843702, 0.7052639895253977, -0.8004

Action: 25, State: ([-2.0971344452485763, 1.6512339076397042, -2.0843296487167837, 0.7354950807774043, -0.8291133689102226, -0.3912648014406761, -0.13838933408410817, 0.2556968329898747, -0.027397437819737414, -0.21041570834617976, -0.09389647633980201, 0.4221686898498746],), Reward: -13.4515052046
Action: 25, State: ([-2.0972246071019693, 1.6512067106205066, -2.0836469693664803, 0.7295049456096377, -0.8434737518477595, -0.38793331504527445, -0.13882741955160735, 0.25568838874100075, -0.027713158145649675, -0.21041570834617976, -0.09389647633980201, 0.4221686898498746],), Reward: -13.4517354406
Action: 25, State: ([-2.097236527028821, 1.6517256076048428, -2.083342207558137, 0.7405785442293661, -0.8207822916327874, -0.32711716916442857, -0.13884779014563517, 0.2558633299858605, -0.027549796474929, -0.21041570834617976, -0.09389647633980201, 0.4221686898498746],), Reward: -13.4518827367
Action: 25, State: ([-2.0973485971400656, 1.6510743752259325, -2.0849522116143557, 0.7374712942579873,

KeyboardInterrupt: 

In [50]:
env._actions[25]

(array([-2.0943951,  2.0943951, -2.0943951]),)